# Yahoo Finance quotes

We'd like to get the latest oil price. We can use Yahoo Finance. There are two ways: get CSV data from `download.finance.yahoo.com` or use `query.yahooapis.com` to do the same thing but parse it into JSON.

## Quick and dirty CSV

[Here is an explanation](http://mdbitz.com/2010/02/02/understanding-yahoo-finance-stock-quotes-and-sl1d1t1c1ohgv/) of the `f` parameter.

In [1]:
import requests

symbol = 'CLF17.NYM'
url = "http://download.finance.yahoo.com/d/quotes.csv"
params = {'s': symbol, 'f': 'l1'}

r = requests.get(url, params=params)

In [2]:
try:
    print(float(r.text))
except ValueError:
    print(r.text)

48.02


In [3]:
# If we get more data, eg f: sl1d1t1 then we can
# import csv
# data = list(csv.reader([r.text]))[0]

In [4]:
# data

Benchmarks that seem to work with this service:

* CL — West Texas Intermediate or WTI, light sweet crude
* BB — Brent crude penultimate financial futures
* BZ — Brent look-alike crude oil futures
* MB — Gulf Coast Sour Crude
* RE — Russian Export Blend Crude Oil (REBCO) futures

Gas spot prices that work:

* NG — Henry Hub physical futures
* HH — Henry Hub last day financial futures

Symbols that don't work:

* DC — Dubai crude calendar futures
* WCC — Canadian Heavy (differential, cf CL)
* WCE — Western Canadian select crude oil futures (differential, cf CL) — but this seems to be the same price as WCC, which can't be right
* LN — European options

## Parse into JSON

This does more or less the same thing, but we wrap it in another service to get at the JSON. We might do this when things start to get more complicated, like...

In [5]:
symbol = 'BZX16.NYM'
url = "http://download.finance.yahoo.com/d/quotes.csv"
params = {'s': symbol, 'f': 'sl1d1t1'}

r = requests.get(url, params=params)
r.text

'"BZX16.NYM",48.54,"9/7/2016","6:03pm"\n'

We'll use the url that `requests` formed, above:

In [6]:
r.url

'http://download.finance.yahoo.com/d/quotes.csv?f=sl1d1t1&s=BZX16.NYM'

In [7]:
query = "select * from csv"
query += " where url='{}' and".format(r.url)
query += " columns='symbol,price,data,time'"  # corresponds to f parameter

In [8]:
url = "https://query.yahooapis.com/v1/public/yql"

In [9]:
env = "http://datatables.org/alltableswithkeys.env"

In [10]:
params = {
    'q': query,
    'format': 'json',
    'env': env,
}

In [11]:
r = requests.get(url, params=params)

In [12]:
j = r.json()

In [13]:
j

{'query': {'count': 1,
  'created': '2016-09-07T22:30:39Z',
  'lang': 'en-US',
  'results': {'row': {'data': '9/7/2016',
    'price': '48.54',
    'symbol': 'BZX16.NYM',
    'time': '6:03pm'}}}}

In [14]:
price = float(j['query']['results']['row']['price'])

In [15]:
price

48.54

## Forming symbols

What's the symbol? Crude futures symbols are funky, e.g.

* CL — West Texas Intermediate or WTI, light sweet crude
* BZ — Brent look-alike crude oil futures
* MB — Gulf Coast Sour Crude
* BB — Brent crude penultimate financial futures
* NG — Henry Hub physical futures
* HH — Henry Hub last day financial futures

The ticker symbols we're passing look like XXMYY.NYM, with components as follows:

* `XX` — commodity symbol, as above: CL, BB, etc.
* `M` — a month code, symbolizing January to December: `[F,G,H,J,K,M,N,Q,U,V,X,Z]`
* `YY` — a two-digit year, like 13.
* `.NYM` — the Nymex symbol.

In [16]:
import time

def get_symbol(benchmark):
    # We compute a time 45 days in the future for a price
    future = time.gmtime(time.time() + 45*24*60*60)
    month = future.tm_mon
    month_codes = ['F', 'G', 'H', 'J', 'K', 'M', 'N', 'Q', 'U', 'V', 'X', 'Z']
    month = month_codes[month - 1]
    year = str(future.tm_year)[-2:]

    symbol = benchmark + month + year + ".NYM"
    return symbol

In [17]:
get_symbol('WCC')

'WCCV16.NYM'